# 아마존 세이지메이커에 Idefics 9B & 80B 모델 배포하기

IDEFICS는 800억 개의 매개변수를 가진 오픈 액세스 시각적 언어 모델로, 이미지와 텍스트 시퀀스를 기반으로 텍스트를 생성할 수 있습니다. 이 모델은 공개적으로 이용 가능한 데이터와 모델만 사용하여 Deepmind의 폐쇄형 Flamingo 또는 OpenAI의 GPT-4V 모델과 유사한 기능을 재현하기 위해 만들어졌습니다.

In this blog you will learn how to deploy Idefics model to Amazon SageMaker. We are going to use the Hugging Face LLM DLC is a new purpose-built Inference Container to easily deploy LLMs in a secure and managed environment. The DLC is powered by [Text Generation Inference (TGI)](https://github.com/huggingface/text-generation-inference) a scalelable, optimized solution for deploying and serving Large Language Models (LLMs). The Blog post also includes Hardware requirements for the different model sizes. 

이 노트북에서는 IDEFICS 모델을 아마존 세이지메이커에 배포하는 방법을 배웁니다. 우리는 새로운 목적으로 구축된 추론 컨테이너인 허깅 페이스 LLM DLC를 사용하여 안전하고 관리되는 환경에서 LLM을 쉽게 배포할 것입니다. DLC는 대규모 언어 모델(LLM)을 배포하고 서비스하기 위한 확장 가능하고 최적화된 솔루션인 텍스트 생성 추론(Text Generation Inference; TGI)에 의해 구동됩니다. 이 노트북에는 다양한 모델 크기에 대한 하드웨어 요구 사항도 포함되어 있습니다.

이 노트북은 다음 내용을 다룹니다:
1. [개발 환경 설정](#1-개발-환경-설정)
2. [새로운 허깅페이스 LLM DLC 가져오기](#2-새로운-허깅페이스-LLM-DLC-가져오기)
3. [하드웨어 요구사항](#3-하드웨어-요구사항)
4. [Idefics 80B 모델을 아마존 세이지메이커에 배포하기](#4-Idefics-80B-모델을-아마존-세이지메이커에-배포하기)
5. [추론 실행 및 모델과 대화하기](#5-추론-실행-및-모델과-대화하기)
6. [정리 작업](#6-정리-작업)

시작하겠습니다!


## 1. 개발 환경 설정

`sagemaker` 파이썬 SDK를 사용하여 Idefics를 아마존 세이지메이커에 배포합니다. AWS 계정이 구성되어 있고 `sagemaker` 파이썬 SDK가 설치되어 있는지 확인해야 합니다.

In [2]:
!pip install "sagemaker>=2.192.0" --upgrade --quiet

로컬 환경에서 세이지메이커를 사용하려면 세이지메이커에 필요한 권한이 있는 IAM 역할에 대한 접근 권한이 필요합니다. 자세한 내용은 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)에서 확인할 수 있습니다.

In [3]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker 세션 버킷 -> 데이터, 모델 및 로그 업로드에 사용
# 버킷이 존재하지 않으면 sagemaker가 자동으로 버킷을 생성합니다.
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # 버킷 이름이 주어지지 않으면 기본 버킷으로 설정합니다.
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::079002598131:role/service-role/AmazonSageMaker-ExecutionRole-20220804T150518
sagemaker session region: us-west-2


## 2. 새로운 허깅페이스 LLM DLC 가져오기

일반적인 허깅 페이스 모델을 배포하는 것과 비교하여, 먼저 컨테이너 URI를 검색하고 이를 `image_uri`로 지정하여 `HuggingFaceModel` 모델 클래스에 제공해야 합니다. 아마존 세이지메이커에서 새로운 허깅 페이스 LLM DLC를 검색하려면 `sagemaker` SDK에서 제공하는 `get_huggingface_llm_image_uri` 메서드를 사용할 수 있습니다. 이 메서드를 통해 지정된 `backend`, `session`, `region` 및 `version`에 따라 원하는 허깅페이스 LLM DLC의 URI를 검색할 수 있습니다. 사용 가능한 버전은 [여기](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#huggingface-text-generation-inference-containers)에서 확인할 수 있습니다.


In [4]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# llm 이미지 URI를 검색합니다.
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="1.1.0"
)

# ECR 이미지 URI를 출력합니다.
print(f"llm image uri: {llm_image}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
llm image uri: 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04


## 3. 하드웨어 요구사항

Idefics comes in 2 different sizes - 9B & 80B parameters. The hardware requirements will vary based on the model size deployed to SageMaker. Below is a set up minimum requirements for each model size we tested.

Idefics는 90억 매개변수 버전와 800억 매개변수 버전 두 가지로 제공됩니다. 세이지메이커에 배포되는 모델 크기에 따라 하드웨어 요구사항이 달라집니다. 아래는 각 모델 크기에 대해 테스트한 최소 요구사항입니다. 

_참고: 아직 GPTQ 모델은 테스트하지 않았습니다._


| 모델                                                              | 인스턴스 유형             | 양자화            | 복제본당 GPU 수 | 
|-----------------------------------------------------------------|---------------------|----------------|------------|
| [Idefics 9B](https://huggingface.co/HuggingFaceM4/idefics-9b)   | `(ml.)g5.12xlarge`  | `-`            | 4          |                  | 
| [Idefics 80B](https://huggingface.co/HuggingFaceM4/idefics-80b) | `(ml.)g5.48xlarge`  | `bitsandbytes` | 8          | 
| [Idefics 80B](https://huggingface.co/HuggingFaceM4/idefics-80b) | `(ml.)p4d.24xlarge` | `-`            | 8          | 

_참고: 아마존 세이지메이커는 현재 인스턴스 슬라이싱을 지원하지 않습니다. 예를 들어, Idefics 800억 매개변수 버전을 위해 단일 인스턴스에서 여러 복제를 실행할 수 없습니다._

다음은 Idefics instruct 90억 및 800억 모델이 세이지메이커에서 작동하도록 검증된 설정입니다.


## 4. Idefics 80B 모델을 아마존 세이지메이커에 배포하기

[HuggingFaceM4/idefics-80b-instruct](https://huggingface.co/HuggingFaceM4/idefics-80b-instruct)모델을 아마존 세이지메이커에 배포하기 위해 `HuggingFaceModel` 모델 클래스를 생성하고 `hf_model_id`, `instance_type` 등 엔드포인트 설정을 정의합니다. 실습에서는 4개의 NVIDIA A10G GPU와 96GB의 GPU 메모리를 가진 `ml.g5.12xlarge` 인스턴스 유형을 사용합니다.


In [5]:
import json
from sagemaker.huggingface import HuggingFaceModel

# 세이지메이커 설정
instance_type = "ml.g5.12xlarge"
number_of_gpu = 4
health_check_timeout = 600

# 모델 및 엔드포인트 설정 매개변수 정의
config = {
  'HF_MODEL_ID': "HuggingFaceM4/idefics-9b-instruct", # hf.co/models의 모델 ID
  'SM_NUM_GPUS': json.dumps(number_of_gpu), # 복제본당 사용되는 GPU 수
  'MAX_INPUT_LENGTH': json.dumps(1024),  # 입력 텍스트의 최대 길이
  'MAX_TOTAL_TOKENS': json.dumps(2048),  # 생성되는 텍스트의 최대 길이 (입력 텍스트 포함)
  'MAX_BATCH_TOTAL_TOKENS': json.dumps(8192),  # 생성 중 병렬로 처리할 수 있는 토큰 수 제한
  'HF_MODEL_QUANTIZE': "bitsandbytes", # 양자화 방식
}

# 이미지 URI로 HuggingFaceModel 생성하기
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  env=config
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


`HuggingFaceModel`을 생성한 후, `deploy` 메서드를 사용하여 아마존 세이지메이커에 배포할 수 있습니다. 실습에서는 `ml.g5.12xlarge` 인스턴스 유형으로 모델을 배포합니다. TGI는 자동으로 모델을 모든 GPU에 분산하고 샤딩합니다.

In [6]:
# 엔드포인트에 모델 배포하기
# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy
llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout, # 모델을 적재하는데 10분 정도 소요됩니다.
)


--------!

이제 세이지메이커는 엔드포인트를 생성하고 모델을 배포합니다. 이 과정은 10-15분 정도 소요될 수 있습니다.

## 5. 추론 실행 및 모델과 대화하기

엔드포인트가 배포된 후에는 해당 엔드포인트에서 추론을 실행할 수 있습니다. 엔드포인트에서 추론을 실행하기 위해 `predictor`의 `predict` 메서드를 사용합니다. 생성에 영향을 미치는 다양한 매개변수를 사용하여 추론할 수 있습니다. 매개변수는 페이로드의 `parameters` 속성에 정의할 수 있습니다. 매개변수의 전체 목록은 [문서](https://huggingface.github.io/text-generation-inference/) 하단의 GenerateParameters 객체에서 확인할 수 있습니다.


`HuggingFaceM4/idefics-80b-instruct`는 인스터럭션 기반으로 조정된 모델이므로 다음 프롬프트를 사용하여 지시할 수 있습니다:
  
```
User:<fake_token_around_image><image><fake_token_around_image>{in_context_prompt}<end_of_utterance>\n
Assistant: {in_context_answer}<end_of_utterance>\n
User:<fake_token_around_image><image><fake_token_around_image>{prompt}<end_of_utterance>\n
Assistant:
```

TGI에 대해 더 자세한 내용은 [여기](https://github.com/huggingface/transformers/issues/25803)를 참고하세요. 현재 TGI에서는 `<image>`를 "markdown" URL 형식으로 추가해야 하며 `![](https://t4.ftcdn.net/jpg/00/97/58/97/360_F_97589769_t45CqXyzjz0KXwoBZT9PRaWGHRk5hQqQ.jpg)`와 같이 입력합니다. TGI 1.1.0 버전에서는 이미지 URL만 제공할 수 있습니다. 안전한 추론을 실행하고 로컬 이미지를 사용하기 위해 이미지를 s3에 업로드하고 서명된 URL을 통해 사용할 수 있도록 한 후 추론이 끝나면 다시 삭제할 것입니다. 이를 위해 프롬프트와 이미지 경로를 받아들이는 헬퍼 메서드 `run_inference`를 생성합니다. 이미지는 s3에 업로드되고 서명된 URL이 생성됩니다. 그런 다음 엔드포인트에서 추론을 실행하고 이미지를 다시 삭제합니다.

In [7]:
from botocore.client import Config

s3 = sess.boto_session.client('s3', config=Config(signature_version='s3v4'))

prompt_template="User:{prompt}![]({image})<end_of_utterance>\nAssistant:"
parameters = {
    "do_sample": True,
    "top_p": 0.2,
    "temperature": 0.4,
    "top_k": 50,
    "max_new_tokens": 512,
    "stop": ["User:","<end_of_utterance>"]
  }

def run_inference(prompt=None,image_path=None):
    # 매개변수
    bucket = sess.default_bucket()
    key = os.path.join("input", os.path.basename(image_path))
    
    # S3에 이미지 업로드하기    
    s3.upload_file(image_path, bucket, key)

    # 5분 동안 유효한 서명된 URL 생성
    url = s3.generate_presigned_url(
        ClientMethod='get_object', 
        Params={'Bucket': bucket, 'Key': key},
        ExpiresIn=300
    )
    # 이미지와 텍스트를 포함한 프롬프트 생성
    parsed_prompt = prompt_template.format(image=url,prompt=prompt)

    # 프롬프트와 매개변수를 포함하여 엔드포인트 호출
    chat = llm.predict({"inputs":parsed_prompt,"parameters":parameters})

    # S3에서 이미지 삭제
    s3.delete_object(Bucket=bucket, Key=key)
    
    # 생성된 텍스트 반환
    return chat[0]["generated_text"][len(parsed_prompt):].strip()

인터넷에서 요청에 사용할 이미지를 가져옵니다. 다음 이미지를 사용합니다:

![power cord](img/powercord.jpeg). 

"if i can use this power cord in the U.S?(이 전원 코드를 미국에서 사용할 수 있나요?)"라고 물어봅시다.

In [12]:
import os

prompt = "Can I use this cable in the U.S.?"

# 추론 수행
res = run_inference(prompt=prompt,
                    image_path="img/powercord.jpeg")

print(res)
# 아니요, 이 케이블은 미국 콘센트와 호환되지 않습니다. 이 케이블은 두 개의 둥근 핀이 있는 유럽형 플러그로, 두 개의 평평한 핀이 있는 미국 표준 콘센트와 호환되지 않습니다.

No, this cable is not suitable for use in the U.S.


정답은 이 케이블은 유럽 케이블로 미국에서는 사용할 수 없다는 것입니다.

## 6. 정리 작업

모델과 엔드포인트를 삭제하여 정리합니다.


In [ ]:
# llm.delete_model()
# llm.delete_endpoint()